# Dekorátory

In [ ]:
# Vynucení kontroly souladu s PEP8
!pip install flake8 pycodestyle pycodestyle_magic
%load_ext pycodestyle_magic
%pycodestyle_on

### 1. Vytvořte dekorátor, který při zavolání funkce vypíše (na konzoli) jméno volané funkce, přehled vstupních parametrů a celkový čas, který byl potřeba k jejímu vykonání.

**Hint:** Čas můžete změřit pomocí metody `time.time()` – ta vrací čas v sekundách od začátku epochy v okamžiku svého zavolání.

In [ ]:
import time


def info(funkce):
    """Informační dekorátor."""
    def wrapper(*args, **kwargs):
        # výpis vlastností funkce
        print(
            'Volána funkce "{}".'.format(funkce.__name__),
            'Její poziční argumenty: {}'.format(args),
            'Její pojmenované argumenty: {}'.format(kwargs),
            sep='\n'
        )
        # obsluha měřiče času
        start = time.time()
        # zavolání původní funkce
        ret = funkce(*args, **kwargs)
        # výpis času potřebného pro její vykonání
        print(
            'Vykonání funkce trvalo {} sekundy.'.format(time.time() - start)
        )
        print()
        return ret
    return wrapper

In [ ]:
@info
def fn1(a, b, zaokrouhlení=None):
    výsledek = a**b
    return výsledek


print(' fn1(3, 4) ')
fn1(3, 4)
print(' fn1(3, 4, zaokrouhlení="test") ')
fn1(3, 4, zaokrouhlení="test")


@info
def fn2(a, b, zaokrouhlení=None):
    import time
    time.sleep(3)
    výsledek = a**b
    return výsledek


print(' fn2(3, 4) ')
fn2(3, 4)
print(' fn2(3, 4, zaokrouhlení="testík") ')
fn2(3, 4, zaokrouhlení="testík")

### 2. Vytvořte dekorátor, který volané funkci pošle vstupní poziční parametry v opačném pořadí, než byly zadány.

**Hint:** Nezapomeňte cestou na pojmenované argumenty.

In [ ]:
def reverse_args(funkce):
    """Zamění pořadí vstupních pozičních argumentů za opačné."""
    def wrapper(*args, **kwargs):
        reversed_args = reversed(args)
        return funkce(*reversed_args, **kwargs)
    return wrapper

In [ ]:
@reverse_args
def fn(a, b, c):
    return a, b, c


xs = 3, 4, 5
print(xs, '=>', fn(*xs))
xs = 'a', 'b', 'c'
print(xs, '=>', fn(*xs))

### 3. Vytvořte dekorátor, který pro funkci, jejíž výstupní hodnotou bude seznam, vrátí tento seznam seřazený.

**Hint:** Uvažujte implicitní řazení `sorted()`.

In [ ]:
def sort_return_list(funkce):
    """Výstupní seznam (ale pouze seznam) setřídí."""
    def wrapper(*args, **kwargs):
        ret = funkce(*args, **kwargs)
        if isinstance(ret, list):
            return sorted(ret)
        else:
            return ret
    return wrapper

In [ ]:
@sort_return_list
def fn(xs):
    return xs


xs = (3, 5, 4)
print(xs, '=>', fn(xs))
xs = [3, 5, 4]
print(xs, '=>', fn(xs))

xs = {'c', 'b', 'a'}
print(xs, '=>', fn(xs))
xs = ['c', 'b', 'a']
print(xs, '=>', fn(xs))

### 4. Upravte předchozí dekorátor tak, aby na svém vstupu přebíral předpis pro třídění. Tj. jedná se o dekorátor s parametrem, kterým je hodnota parametru key pro funkci sorted().

**Hint:** Jedná se tedy vlastně o dva dekorátory zanořené v sobě (podívejte se do přednášek), kde ten vnější z nich přebírá příslušnou hodnotu pro atribut key (buď odkaz na existující funkci nebo přímo vlastní lambda-funkci) a teprve druhý, vnitřní vlastní dekorovanou funkci.

In [ ]:
def sort_return_list_with_key(keypar):
    """Výstupní seznam (ale pouze seznam) setřídí podle dodaného klíče."""
    def decorator_wrapper(funkce):
        def wrapper(*args, **kwargs):
            ret = funkce(*args, **kwargs)
            if isinstance(ret, list):
                return sorted(ret, key=keypar)
            else:
                return ret
        return wrapper
    return decorator_wrapper

In [ ]:
@sort_return_list_with_key(lambda x: x[0])
def fn1(xs):
    return xs


@sort_return_list_with_key(lambda x: x[1])
def fn2(xs):
    return xs


xs = [(1, 3), (2, 2), (3, 1)]
print(xs, '=>', fn1(xs))
print(xs, '=>', fn2(xs))

xs = {(1, 3), (2, 2), (3, 1)}
print(xs, '=>', fn1(xs))
print(xs, '=>', fn2(xs))

### 5. Vytvořte dekorátor, který pro funkci vracející fragmenty HTML odstraní z tohoto řetězce veškeré formátovací značky.

Tj. například pro vstup `<p class='test'>ahojte</p>` vrátí `ahojte`.

**Hint**: Modul `re`.

In [ ]:
import re


def dehtmlize(funkce):
    """Odstraňuje z výstupního textu HTML-značky."""
    def wrapper(*args, **kwargs):
        ret = funkce(*args, **kwargs)
        if isinstance(ret, str):
            pattern = re.compile(r'</?.*?>', re.IGNORECASE)
            return pattern.sub('', ret)
        else:
            return ret
    return wrapper

In [ ]:
@dehtmlize
def fn(xs):
    return xs


xs = (3, 5, 4)
print(xs, '=>', fn(xs))
xs = {3, 5, 4}
print(xs, '=>', fn(xs))
xs = "Ahoj, světe!"
print(xs, '=>', fn(xs))
xs = "<span class='baf'>Ahoj</span>, <em>světe</em>!"
print(xs, '=>', fn(xs))
xs = "<span class=\"baf\">Ahoj</span>, <em>s<u>věte</u>!</em>"
print(xs, '=>', fn(xs))